<img align="left" src = figs/logos/logo-IJCLab_v1.png width=250, style="padding: 10px"> 
<b>Find BAO in DC2 </b> <br>
Last verified to run on 2022-01-30 with LSST Science Pipelines release w_2021_49 <br>
Contact authors: Sylvie Dagoret-Campagne (DP0 Delegate) <br>
Target audience: DP0 delegates member <br>

**Credit:** Originally developed by Sylvie Dagoret-Campagne in the framework provided by Rubin DP0.1 (reference DP0.1 tutorials)

Acknowledgement: Melissa Graham, Leanne Guy, Alex Drlica-Wagner, Keith Bechtol, Grzegorz Madejski, Louise Edwards, and many others ..

### Learning Objectives

The goal is to detect BAO inside DC2

- inspired from https://www.youtube.com/watch?v=HUUoX90MW7w&list=PLnq2BjRWgmjv1ZpMQO2iLdZR0sVWP7AqK

### Imports

In [ ]:
# Import general python packages
import numpy as np
import re
import pandas as pd
import pickle
from pandas.testing import assert_frame_equal
import os
import errno
import shutil
import getpass

# Import the Rubin TAP service utilities
from lsst.rsp import get_tap_service, retrieve_query

# LSST Science Pipelines (Stack) packages
import lsst.daf.butler as dafButler
import lsst.afw.display as afwDisplay
import lsst.geom as geom
import lsst.afw.coord as afwCoord
afwDisplay.setDefaultBackend('matplotlib')

#
from lsst import skymap

# Astropy
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.units.quantity import Quantity
from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch


# Bokeh for interactive visualization
import bokeh
from bokeh.io import output_file, output_notebook, show
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource, CDSView, GroupFilter, HoverTool
from bokeh.plotting import figure
from bokeh.transform import factor_cmap

import holoviews as hv
from holoviews import streams, opts
from holoviews.operation.datashader import rasterize
from holoviews.operation.datashader import datashade, dynspread
from holoviews.plotting.util import process_cmap

import datashader as dsh


# Set the maximum number of rows to display from pandas
pd.set_option('display.max_rows', 20)


# Set the holoviews plotting library to be bokeh
# You will see the holoviews + bokeh icons displayed when the library is loaded successfully
#hv.extension('bokeh')
hv.extension('bokeh', 'matplotlib')

# Display bokeh plots inline in the notebook
output_notebook()

In [ ]:
# What versions of bokeh and holoviews nd datashader are we working with?
# This is important when referring to online documentation as
# APIs can change between versions.
print("Bokeh version: " + bokeh.__version__)
print("Holoviews version: " + hv.__version__)
print("Datashader version: " + dsh.__version__)

In [ ]:
# allow for matplotlib to create inline plots in our notebook
%matplotlib inline
import matplotlib.pyplot as plt      # imports matplotlib.pyplot as plt
from matplotlib.colors import Normalize

import warnings                      # imports the warnings library
import gc                            # imports python's garbage collector

# Ignore warnings
from astropy.units import UnitsWarning
warnings.simplefilter("ignore", category=UnitsWarning)

In [ ]:
# Set up some plotting defaults:

params = {'axes.labelsize': 28,
          'font.size': 24,
          'legend.fontsize': 14,
          'xtick.major.width': 3,
          'xtick.minor.width': 2,
          'xtick.major.size': 12,
          'xtick.minor.size': 6,
          'xtick.direction': 'in',
          'xtick.top': True,
          'lines.linewidth': 3,
          'axes.linewidth': 3,
          'axes.labelweight': 3,
          'axes.titleweight': 3,
          'ytick.major.width': 3,
          'ytick.minor.width': 2,
          'ytick.major.size': 12,
          'ytick.minor.size': 6,
          'ytick.direction': 'in',
          'ytick.right': True,
          'figure.figsize': [18, 10],
          'figure.facecolor': 'White'
          }

plt.rcParams.update(params)

In [ ]:
from astropy.cosmology import FlatLambdaCDM, Planck18

In [ ]:
from IPython.display import Image

In [ ]:
def remove_figure(fig):
    """Remove a figure to reduce memory footprint. """
    # get the axes and clear their images
    for ax in fig.get_axes():
        for im in ax.get_images():
            im.remove()
    fig.clf()      # clear the figure
    plt.close(fig) # close the figure
    gc.collect()   # call the garbage collector

In [ ]:
# What version of the Stack are we using?
! echo $IMAGE_DESCRIPTION
! eups list -s | grep lsst_distrib

In [ ]:
import numpy as np
import numexpr

class ImageStructureFactor:
    """A class to compute rapially averaged structure factor of a 2D image"""
    def __init__(self, shape):
        assert len(shape) == 2, "only 2D images implemented"
        L = max(shape)
        self.qs = np.fft.fftfreq(L)[:int(L/2)]
        self.dists = np.sqrt(np.fft.fftfreq(shape[0])[:,None]**2 + np.fft.fftfreq(shape[1])**2)
        self.dcount = np.histogram(self.dists.ravel(), bins=self.qs)[0]
        self.has_window = False

    def set_window(self,w='hanning'):
        if w == False:
            self.has_window = False
        elif hasattr(np, w):
            self.window = [getattr(np,w)(self.dists.shape[0])[:,None], getattr(np,w)(self.dists.shape[1])]
            self.has_window = True
        elif isinstance(w, np.ndarray):
            assert w.shape == self.dists.shape
            self.window = w
            self.has_window = True

    def windowing(self, im):
        if not self.has_window:
            return im
        if isinstance(self.window, np.ndarray):
            return numexpr.evaluate('im*w', {'im':im, 'w':self.window})
        return numexpr.evaluate('im*w0*w1', {'im':im, 'w0':self.window[0], 'w1':self.window[1]})

    def __call__(self, im):
        spectrum = numexpr.evaluate(
            'real(abs(f))**2',
            {'f':np.fft.fft2(self.windowing(im))}
            )
        return np.histogram(self.dists.ravel(), bins=self.qs, weights=spectrum.ravel())[0] / self.dcount


### 1. Notebook Configuration

#### 1.1 setup pathes

In [ ]:
# username
myusername=getpass.getuser()

In [ ]:
# temporary folders if necessary
NBDIR       = 'bao_lss02'                             # relative path for this notebook output
TMPTOPDIR   = "/scratch"                               # always write some output in /scratch, never in user HOME 
TMPUSERDIR  = os.path.join(TMPTOPDIR,myusername)       # defines the path of user outputs in /scratch 
TMPNBDIR    = os.path.join(TMPUSERDIR,NBDIR)           # output path for this particular notebook

In [ ]:
# create user temporary directory
if not os.path.isdir(TMPUSERDIR):
    try:
        os.mkdir(TMPUSERDIR)
    except:
        raise OSError(f"Can't create destination directory {TMPUSERDIR}!" ) 

In [ ]:
# create this notebook temporary directory
if not os.path.isdir(TMPNBDIR):
    try:
        os.mkdir(TMPNBDIR)
    except:
        raise OSError(f"Can't create destination directory {TMPNBDIR}!" ) 

#### 1.2 Defines steering flags and parameters

The Output queries may be saved in files if requested. 
By defaults all the following flags are set False : no query output is saved in file.
To speed-up the demo, the presenter may keep some of those flags True.


In [ ]:
FLAG_WRITE_DATAFRAMEONDISK  = True                     # Select of query output will be saved on disk
FLAG_READ_DATAFRAMEFROMDISK = True                     # Select of the query can be red from disk if it exists
FLAG_CLEAN_DATAONDISK       = False                     # Select of the output queries saved in file will be cleaned at the end of the notebook

### 2. Get a slice

In [ ]:
# Define a function to build a query passing a coordinate and a search radius
def getQueryCircle(c: SkyCoord, r: Quantity) -> str:
    query = "SELECT obj.ra, obj.dec, obj.objectId, obj.extendedness, "\
            "truth_type, truth.redshift, truth.match_objectId " \
            "FROM dp01_dc2_catalogs.object as obj " \
            "JOIN dp01_dc2_catalogs.truth_match as truth " \
            "ON truth.match_objectId = obj.objectId " \
            "WHERE CONTAINS(POINT('ICRS', obj.ra, obj.dec),"\
            "CIRCLE('ICRS', " + str(c.ra.value) + ", " + str(c.dec.value) + ", " \
            + str(r.to(u.deg).value) + " )) = 1 " \
            "AND obj.good = 1 "  \
            "AND truth.match_objectid >= 0 " \
            "AND truth.is_good_match = 1"
    return query

In [ ]:
# Define a function to build a query passing a coordinate and a search radius
def getQuerySquare(c: SkyCoord, r: Quantity) -> str:
    
    line_ra = ""
    line_dec = ""
    
    if c.ra.value >=0:
        line_ra =  "WHERE ABS(" +  "obj.ra" + " - " + str(c.ra.value)  + ") <" + str(r.to(u.deg).value)
    else:
        line_ra =  "WHERE ABS(" +  "obj.ra" + " + " + str(-c.ra.value)  + ") <" + str(r.to(u.deg).value)
  

    if c.dec.value >=0:
        line_dec =  "AND ABS("   + "obj.dec" + " - " + str(c.dec.value) + ") <" + str(r.to(u.deg).value) 
    else:
        line_dec =  "AND ABS("   + "obj.dec" + " + " + str(-c.dec.value) + ") <" + str(r.to(u.deg).value)  
            
        

    query = "SELECT obj.ra, obj.dec, obj.objectId, obj.extendedness, "\
            "truth_type, truth.redshift, truth.match_objectId " \
            "FROM dp01_dc2_catalogs.object as obj " \
            "JOIN dp01_dc2_catalogs.truth_match as truth " \
            "ON truth.match_objectId = obj.objectId " \
            "WHERE ABS(" +  "obj.ra" + " - " + str(c.ra.value)  + ") <" + str(r.to(u.deg).value) + " "\
            "AND ABS("   + "obj.dec" + " + " + str(-c.dec.value) + ") <" + str(r.to(u.deg).value) + " "\
            "AND obj.good = 1 "  \
            "AND truth.match_objectid >= 0 " \
            "AND truth.is_good_match = 1"
    
    
    
    
    return query

#### 2.1 Create the Rubin TAP Service client

In [ ]:
# Get an instance of the TAP service
service = get_tap_service()
assert service is not None
assert service.baseurl == "https://data.lsst.cloud/api/tap"

In [ ]:
# Define a reference position on the sky and a radius in degrees for a cone search
c1 = SkyCoord(ra=55.65*u.degree, dec=-40.*u.degree, frame='icrs')
radius = 4 * u.deg

In [ ]:
query = getQuerySquare(c1, radius)

In [ ]:
query

#### 2.2 Get a full big slice in angle

In [ ]:
filename_result=f'bao_result.pkl'
fullfilename_result=os.path.join(TMPNBDIR,filename_result)

In [ ]:
if FLAG_READ_DATAFRAMEFROMDISK and os.path.exists(fullfilename_result):
    sql_result = pd.read_pickle(fullfilename_result)
else:
    job = service.submit_job(query)
    job.run()
    job.wait(phases=['COMPLETED', 'ERROR'])
    print('Job phase is', job.phase)
    #sql_result = job.fetch_result().to_table().to_pandas()
    
    
    # Create and submit the job. This step does not run the query yet
    job = service.submit_job(query)
    # Get the job URL
    print('Job URL is', job.url)

    # Get the job phase. It will be pending as we have not yet started the job
    print('Job phase is', job.phase)
    
    # Run the job. You will see that the the cell completes executing,
    # even though the query is still running
    job.run()
    
    # Use this to tell python to wait for the job to finish if
    # you don't want to run anything else while waiting
    # The cell will continue executing until the job is finished
    job.wait(phases=['COMPLETED', 'ERROR'])
    print('Job phase is', job.phase)
    
    # A usefull funtion to raise an exception if there was a problem with the query
    job.raise_if_error()
    
    # Once the job completes successfully, you can fetch the results
    async_data = job.fetch_result()
    
    sql_result = async_data.to_table().to_pandas()
    
    
if FLAG_WRITE_DATAFRAMEONDISK:
    sql_result.to_pickle(fullfilename_result)
    

In [ ]:
sql_result.head()

In [ ]:
data = sql_result

In [ ]:
! ls -l $TMPNBDIR

In [ ]:
# map truth_type
data['truth_type']=data['truth_type'].map({1: 'galaxy', 2: 'star', 3: 'SNe'})

In [ ]:
data.head()

#### drop objects that are not galaxies

In [ ]:
data.drop(data.loc[data['truth_type'] != 'galaxy' ].index, inplace=True)

In [ ]:
data.head()

#### 4.1 2D view of the patch (RA,DEC)

##### a) 2D histogram view with matplotlib

In [ ]:
x=data["ra"]
y=data["dec"]
z=data["redshift"]
xmin=x.min()
xmax=x.max()
ymin=y.min()
ymax=y.max()
zmin=z.min()
zmax=z.max()

In [ ]:
H, xedges, yedges = np.histogram2d(x, y, bins=(1000, 1000))

In [ ]:
transform = AsinhStretch() + PercentileInterval(99.)
#transform =  PercentileInterval(98.)

In [ ]:
img_opts = dict(height=400, width=450, 
                xaxis="bottom", 
                padding = 0.01, fontsize={'title': '12pt'},
                colorbar=True, toolbar='right', show_grid=True,
                title= f"(HV Image, histo2D)",
                xlabel="RA",
                ylabel="DEC",
                tools=['hover']
               )    

In [ ]:
# With hv.Image, must flip up-down the image initialy
flipHT=np.flipud(H.T)
img=hv.Image(flipHT, bounds=(xmin,ymin,xmax,ymax) ).opts(cmap="jet").opts(**img_opts)

In [ ]:
rasterize(img)

# Cosmology

In [ ]:
#cosmo = FlatLambdaCDM(H0=70, Om0=0.3, Ob0=None)
cosmo = Planck18

In [ ]:
redshift_grid = np.linspace(0,3,100)

In [ ]:
lumin_dist = [cosmo.luminosity_distance(z).value for z in redshift_grid ]
comov_dist = [cosmo.comoving_distance(z).value for z in redshift_grid ]
angular_dist = [cosmo.angular_diameter_distance(z).value for z in redshift_grid ]

In [ ]:
hvcurv_comov=hv.Curve(zip(redshift_grid,comov_dist),label="comoving distance").opts(color='red', width=400,xlabel="redshift",ylabel="distance (MPC)")
hvcurv_lum=hv.Curve(zip(redshift_grid,lumin_dist),label="luminosity distance").opts(color='green', width=400,xlabel="redshift",ylabel="distance (MPC)")
hvcurv_ang=hv.Curve(zip(redshift_grid,angular_dist),label="angular distance").opts(color='blue', width=400,xlabel="redshift",ylabel="distance (MPC)")

In [ ]:
hvcurv_comov  * hvcurv_ang

# Select Redshift

https://github.com/LSSTDESC/DESchool/blob/master/Feb2015/DE_School_Kirkby.pdf

In [ ]:
#ZMIN = 1.17
#ZMAX = 1.23

#ZMIN = 1.19
#ZMAX = 1.21

ZMIN = 1.13
ZMAX = 1.27


ZMEAN=(ZMIN+ZMAX)/2.

In [ ]:
ZMEAN

In [ ]:
data.head()

In [ ]:
cut = (data.redshift > ZMIN) & (data.redshift < ZMAX)

In [ ]:
slice_data = data[cut]

## Cosmological distances

In [ ]:
slice_data_cosmo = slice_data

In [ ]:
slice_data_cosmo["dl"]=cosmo.luminosity_distance(slice_data_cosmo["redshift"])

In [ ]:
slice_data_cosmo["da"]=cosmo.angular_diameter_distance(slice_data_cosmo["redshift"])

In [ ]:
slice_data_cosmo["dc"]=cosmo.comoving_distance(slice_data_cosmo["redshift"])

In [ ]:
slice_data_cosmo["dra"] = slice_data_cosmo["ra"] - slice_data_cosmo["ra"].mean()
slice_data_cosmo["ddec"] = slice_data_cosmo["dec"] - slice_data_cosmo["dec"].mean()

In [ ]:
slice_data_cosmo["dist_ra"] =slice_data_cosmo["da"]*np.tan(slice_data_cosmo["dra"]*np.pi/180)

In [ ]:
slice_data_cosmo["dist_dec"] = slice_data_cosmo["da"]*np.tan(slice_data_cosmo["ddec"]*np.pi/180)

In [ ]:
img_opts = dict(height=400, width=450, 
                xaxis="bottom", 
                padding = 0.01, fontsize={'title': '12pt'},
                colorbar=True, toolbar='right', show_grid=True,
                tools=['hover']
           )    

In [ ]:
X=slice_data_cosmo["dist_ra"]
Y=slice_data_cosmo["dist_dec"]
Z=slice_data_cosmo["dc"]-cosmo.comoving_distance(ZMEAN).value

In [ ]:
print(X.min(),X.max())

In [ ]:
print(Y.min(),Y.max())

In [ ]:
print(Z.min(),Z.max())

In [ ]:
X.shape

In [ ]:
Y.shape

In [ ]:
Z.shape

In [ ]:
NBINX=501
NBINY=501
NBINZ=501

In [ ]:
(nx_bin, xcount) = np.histogram(X, bins=100)
nx_distribution = hv.Histogram(nx_bin, xcount).opts(
    title=f"angular distance ",facecolor='cyan', 
    xlabel='X distance (MPC)', fontscale=1.2,
    height=300, width=300,tools=['hover'])

In [ ]:
(ny_bin, ycount) = np.histogram(Y, bins=100)
ny_distribution = hv.Histogram(ny_bin, ycount).opts(
    title=f"angular distance ",facecolor='cyan', 
    xlabel='Y distance (MPC)', fontscale=1.2,
    height=300, width=300,tools=['hover'])

In [ ]:
(nz_bin, zcount) = np.histogram(Z, bins=100)
nz_distribution = hv.Histogram(nz_bin, zcount).opts(
    title=f"comoving distance ",facecolor='cyan', 
    xlabel='distance (MPC)', fontscale=1.2,
    height=300, width=300,tools=['hover'])

In [ ]:
nx_distribution  + ny_distribution + nz_distribution 

In [ ]:
sample = np.stack((X,Y,Z),axis=1)

In [ ]:
sample.shape

In [ ]:
Lhalf=120
L=2*Lhalf
L # MPC

In [ ]:
delta_k = 2*np.pi/L
delta_k

In [ ]:
del data
gc.collect()

In [ ]:
hist3d=np.histogramdd(sample, bins=250, range=[[-Lhalf,Lhalf], [-Lhalf,Lhalf], [-Lhalf,Lhalf]])

In [ ]:
H, edges  = hist3d

In [ ]:
H.shape

In [ ]:
Hmean = H.mean()
Hmean

In [ ]:
H = (H - Hmean)/Hmean

In [ ]:
H.max()

In [ ]:
H.shape

In [ ]:
bin_ed,counts = np.histogram(H.flatten(),bins=50,range=(-2,20))

In [ ]:
delta_distribution = hv.Histogram(bin_ed, counts).opts(
    title=f"cell counts ",facecolor='blue', 
    xlabel='', fontscale=1.2,
    height=300, width=300,tools=['hover'])

In [ ]:
delta_distribution

## FFT 3d

In [ ]:
H.shape

In [ ]:
npix=H.shape[0]

In [ ]:
fourier_image = np.fft.fftn(H)

In [ ]:
fourier_amplitudes = np.abs(fourier_image)**2

In [ ]:
kfreq = np.fft.fftfreq(npix) * npix

In [ ]:
kfreq3D = np.meshgrid(kfreq, kfreq,kfreq)

In [ ]:
len(kfreq3D)

In [ ]:
knrm = np.sqrt(kfreq3D[0]**2 + kfreq3D[1]**2 + kfreq3D[2]**2)

In [ ]:
knrm = knrm.flatten()
fourier_amplitudes = fourier_amplitudes.flatten()

In [ ]:
kbins = np.arange(0.5, npix//2+1, 1.)

In [ ]:
kvals = 0.5 * (kbins[1:] + kbins[:-1])

In [ ]:
import scipy.stats as stats

Abins, _, _ = stats.binned_statistic(knrm, fourier_amplitudes,
                                     statistic = "mean",
                                     bins = kbins)

In [ ]:
#Abins *= 4./3.*np.pi * (kbins[1:]**3 - kbins[:-1]**3)

In [ ]:
kvals = kvals*delta_k

In [ ]:
hv.Curve(zip(kvals,Abins),"k (MPC^-1)","P(k)").opts(logx=True,logy=True,width=400)

### 6. Clean the output directory 

In [ ]:
if FLAG_CLEAN_DATAONDISK:
    if os.path.isdir(TMPNBDIR):
        try:
            shutil.rmtree(TMPNBDIR)
        except OSError as e:
            print("Error: %s : %s" % (TMPNBDIR, e.strerror)) 